In [3]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_20.csv')
data = data_with_date.drop(columns=['date'])



def time_delayed_cross_correlation(N_i, N_j, max_tau):
    """
    Computes the time-delayed cross-correlation function for two time series N_i and N_j.
    
    Args:
    N_i (numpy array): Time series for node i.
    N_j (numpy array): Time series for node j.
    max_tau (int): Maximum time delay (positive or negative).
    
    Returns:
    cross_corrs (numpy array): Cross-correlation values for each time delay in the range [-max_tau, max_tau].
    time_lags (numpy array): Corresponding time lags.
    """
    n = len(N_i)
    mean_i = np.mean(N_i)
    mean_j = np.mean(N_j)
    
    # Prepare arrays for cross-correlations and time lags
    cross_corrs = []
    time_lags = range(-max_tau, max_tau + 1)
    
    for tau in time_lags:
        if tau < 0:
            shifted_j = np.roll(N_j, tau)  # shift N_j forward (N_j(t+τ))
            valid_indices = np.arange(-tau, n)  # indices where the time shift is valid
        else:
            shifted_i = np.roll(N_i, -tau)  # shift N_i backward (N_i(t-τ))
            valid_indices = np.arange(0, n - tau)  # indices where the time shift is valid

        # Calculate the cross-correlation for the valid time points
        if tau < 0:
            numerator = np.mean(N_i[valid_indices] * shifted_j[valid_indices]) - mean_i * mean_j
            denominator = np.std(N_i[valid_indices]) * np.std(shifted_j[valid_indices])
        else:
            numerator = np.mean(shifted_i[valid_indices] * N_j[valid_indices]) - mean_i * mean_j
            denominator = np.std(shifted_i[valid_indices]) * np.std(N_j[valid_indices])
        
        # Handle the case where the denominator is zero
        if denominator != 0:
            corr = numerator / denominator
        else:
            corr = 0
        
        cross_corrs.append(corr)    
    
    return np.array(cross_corrs), np.array(time_lags) 


#We have a problem because sround 3000 columns is too much, first let's see what happens with a subset
#Now we will make a test for the first 50 columns
time_series_subset = data.iloc[:, :200]

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Let's compute the cross correlation function for each pair of columns in the subset
subset_columns = time_series_subset.columns
#To store results
adjacency_matrix = np.zeros((len(subset_columns), len(subset_columns)))
                            
for i in range(len(subset_columns)):
        for j in range(len(subset_columns)):
            if i != j:
                N_i = time_series_subset.iloc[:,i].values
                N_j = time_series_subset.iloc[:,j].values
                cross_corrs, _ = time_delayed_cross_correlation(N_i, N_j, max_tau)
                #Now we store the results
                max_corr = np.max(cross_corrs)
                if max_corr >= threshold:
                    adjacency_matrix[i,j] = max_corr


adj_data = pd.DataFrame(adjacency_matrix, index=subset_columns, columns=subset_columns)
print(adj_data)


          1001.0    1003.0    1005.0    1007.0    1009.0    1011.0    1013.0  \
1001.0  0.000000  0.850792  0.783337  0.859640  0.879769  0.748210  0.906328   
1003.0  0.850792  0.000000  0.961628  0.964526  0.958348  0.909740  0.944460   
1005.0  0.783337  0.961628  0.000000  0.928717  0.912884  0.916322  0.900017   
1007.0  0.859640  0.964526  0.928717  0.000000  0.947723  0.903281  0.924158   
1009.0  0.879769  0.958348  0.912884  0.947723  0.000000  0.855754  0.936804   
...          ...       ...       ...       ...       ...       ...       ...   
6077.0  0.556566  0.539095  0.621070  0.531150  0.543095  0.522629  0.511127   
6079.0  0.508377  0.527379  0.601322  0.521439  0.525572  0.527956  0.501933   
6081.0  0.469259  0.501077  0.563153  0.532610  0.510174  0.527076  0.507064   
6083.0  0.539697  0.530258  0.595904  0.565548  0.564104  0.527953  0.522171   
6085.0  0.492503  0.537158  0.610902  0.515583  0.477808  0.520825  0.475322   

          1015.0    1017.0    1019.0  .